In [4]:
# Requirements
import os
import numpy as np
import tensorflow as tf
import glob
from tensorflow.keras.preprocessing import image
from tensorflow.keras.layers import Input, Add, Dense, Activation, ZeroPadding2D, BatchNormalization, Flatten, Conv2D, AveragePooling2D, MaxPooling2D, Concatenate
from tensorflow.keras.models import Model
from tensorflow.keras.initializers import random_uniform, glorot_uniform
from skimage import measure
from PIL import Image
from tensorflow.keras.preprocessing.image import load_img, img_to_array

In [2]:
def fractal_dimension(img):
    img = np.array(img.convert('L'))  # Convert to grayscale
    threshold = 128
    binary_img = img > threshold  # Binary image based on threshold

    def boxcount(binary_img, box_size):
        S = np.add.reduceat(
            np.add.reduceat(binary_img, np.arange(0, binary_img.shape[0], box_size), axis=0),
            np.arange(0, binary_img.shape[1], box_size), axis=1)
        return len(np.where(S > 0)[0])

    sizes = [2, 4, 8, 16, 32]
    counts = []
    for size in sizes:
        count = boxcount(binary_img, size)
        counts.append(count)
        #print(f"Size: {size}, Count: {count}")  # Debug output

    # Convert sizes and counts to numpy arrays for processing
    sizes = np.array(sizes)
    counts = np.array(counts)

    # Filter out zero values for log calculation
    non_zero_indices = (sizes > 0) & (counts > 0)
    filtered_sizes = sizes[non_zero_indices]
    filtered_counts = counts[non_zero_indices]

    # Debug output for filtered sizes and counts

    #print(f"Filtered Sizes: {filtered_sizes}, Filtered Counts: {filtered_counts}")

    if filtered_sizes.size > 0 and filtered_counts.size > 0:
        coeffs = np.polyfit(np.log(filtered_sizes), np.log(filtered_counts), 1)
        return -coeffs[0]
    else:
        #print("No valid sizes or counts for fractal dimension calculation.")
        return 0

In [3]:
# Dataset loading
dataset_url = r'E:\S7 - PROJECT WORK\2750'
batch_size = 32
img_height, img_width = 64, 64
validation_split = 0.2
rescale = 1.0 / 255

In [4]:
# Function to load and preprocess images
def load_and_preprocess_image(img_path, target_size=(64, 64)):
    img = load_img(img_path, target_size=target_size)
    img_array = img_to_array(img)
    img_array = img_array / 255.0  # Normalize
    return img_array

In [5]:
# Prepare data for the model
fractal_features = []
image_data = []

In [6]:
# Loop through all images to calculate fractal features
for file in glob.glob(os.path.join(dataset_url, '*/*.jpg')):
    img_array = load_and_preprocess_image(file)  # Get image array
    image_data.append(img_array)
    
    fractal_dim = fractal_dimension(load_img(file))  # Calculate fractal dimension
    fractal_features.append(fractal_dim)

In [7]:
# Convert lists to NumPy arrays
image_data = np.array(image_data)  # Shape: (num_images, height, width, channels)
fractal_features = np.array(fractal_features)  # Shape: (num_images,)

# Reshape fractal_features to be compatible for concatenation
# Here we reshape to (num_images, 1) to prepare for expansion
fractal_features_reshaped = fractal_features.reshape(-1, 1)  # Shape: (num_images, 1)

# Expand fractal features to match the height and width of the image data
# Create an array with the same height and width as the images
# This will create an array of shape (num_images, height, width)
fractal_features_expanded = np.tile(fractal_features_reshaped[:, np.newaxis], (1, image_data.shape[1], image_data.shape[2]))  # Shape: (num_images, height, width)

# Add a new axis to make it (num_images, height, width, 1)
fractal_features_expanded = fractal_features_expanded[:, :, :, np.newaxis]  # Shape: (num_images, height, width, 1)

# Combine image data and fractal features
combined_data = np.concatenate((image_data, fractal_features_expanded), axis=-1)  # Shape: (num_images, height, width, channels + 1)

# Now combined_data can be used for training your model

In [8]:
print(combined_data.shape)

(27000, 64, 64, 4)


In [9]:
def identity_block(X, f, filters, training=True, initializer=glorot_uniform):
    F1, F2, F3 = filters
    X_shortcut = X

    X = Conv2D(filters=F1, kernel_size=1, strides=(1, 1), padding='valid', kernel_initializer=initializer(seed=0))(X)
    X = BatchNormalization(axis=3)(X, training=training)
    X = Activation('relu')(X)

    X = Conv2D(filters=F2, kernel_size=(f, f), strides=(1, 1), padding='same', kernel_initializer=initializer(seed=0))(X)
    X = BatchNormalization(axis=3)(X, training=training)
    X = Activation('relu')(X)

    X = Conv2D(filters=F3, kernel_size=(1, 1), strides=(1, 1), padding='valid', kernel_initializer=initializer(seed=0))(X)
    X = BatchNormalization(axis=3)(X, training=training)

    X = Add()([X_shortcut, X])
    X = Activation('relu')(X)

    return X

In [10]:
def convolutional_block(X, f, filters, s=2, training=True, initializer=glorot_uniform):
    F1, F2, F3 = filters
    X_shortcut = X

    X = Conv2D(filters=F1, kernel_size=1, strides=(s, s), padding='valid', kernel_initializer=initializer(seed=0))(X)
    X = BatchNormalization(axis=3)(X, training=training)
    X = Activation('relu')(X)

    X = Conv2D(F2, (f, f), strides=(1, 1), padding='same', kernel_initializer=initializer(seed=0))(X)
    X = BatchNormalization(axis=3)(X, training=training)
    X = Activation('relu')(X)

    X = Conv2D(F3, (1, 1), strides=(1, 1), padding='valid', kernel_initializer=initializer(seed=0))(X)
    X = BatchNormalization(axis=3)(X, training=training)

    X_shortcut = Conv2D(F3, (1, 1), strides=(s, s), padding='valid', kernel_initializer=initializer(seed=0))(X_shortcut)
    X_shortcut = BatchNormalization(axis=3)(X_shortcut, training=training)

    X = Add()([X, X_shortcut])
    X = Activation('relu')(X)

    return X

In [11]:
def ResNet50(input_shape=(64, 64, 4), classes=10):  # Adjust input shape for fractal features
    X_input = Input(input_shape)

    X = ZeroPadding2D((3, 3))(X_input)
    
    X = Conv2D(64, (7, 7), strides=(2, 2), kernel_initializer=glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis=3)(X)
    X = Activation('relu')(X)
    X = MaxPooling2D((3, 3), strides=(2, 2))(X)

    X = convolutional_block(X, f=3, filters=[64, 64, 256], s=1)
    X = identity_block(X, 3, [64, 64, 256])
    X = identity_block(X, 3, [64, 64, 256])
    
    X = convolutional_block(X, f=3, filters=[128, 128, 512], s=2)
    X = identity_block(X, 3, [128, 128, 512])
    X = identity_block(X, 3, [128, 128, 512])
    X = identity_block(X, 3, [128, 128, 512])

    X = convolutional_block(X, f=3, filters=[256, 256, 1024], s=2)
    X = identity_block(X, 3, [256, 256, 1024])
    X = identity_block(X, 3, [256, 256, 1024])
    X = identity_block(X, 3, [256, 256, 1024])
    X = identity_block(X, 3, [256, 256, 1024])
    X = identity_block(X, 3, [256, 256, 1024])

    X = convolutional_block(X, f=3, filters=[512, 512, 2048], s=2)
    X = identity_block(X, 3, [512, 512, 2048])
    X = identity_block(X, 3, [512, 512, 2048])

    X = AveragePooling2D(pool_size=(2, 2), name='avg_pool')(X)
    
    X = Flatten()(X)
    X = Dense(classes, activation='softmax', kernel_initializer=glorot_uniform(seed=0))(X)
    
    model = Model(inputs=X_input, outputs=X)

    return model

In [12]:
# Compiling the model
model = ResNet50(input_shape=(64, 64, 4), classes=10) 

In [13]:
# Adjusted input shape if adding fractal features
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [14]:
from keras.layers import Input, Conv2D, BatchNormalization, Activation, Add, MaxPooling2D, AveragePooling2D, Flatten, Dense, ZeroPadding2D
from keras.models import Model
from keras.initializers import glorot_uniform
from keras.utils import to_categorical  # Importing to_categorical
from keras.callbacks import EarlyStopping, ModelCheckpoint

In [15]:
# Prepare your dataset (replace this with your actual data)
combined_data = np.random.rand(27000, 64, 64, 4)  # Example placeholder for combined data
num_classes = 10  # Adjust this according to your dataset
labels = np.array([np.random.randint(0, num_classes) for _ in range(27000)])  # Replace with your actual labels
labels_one_hot = to_categorical(labels, num_classes=num_classes)

In [16]:
# Ensure labels shape is correct
assert labels_one_hot.shape[0] == combined_data.shape[0], "Labels and data size mismatch!"

In [17]:
# Callbacks for early stopping and model checkpointing
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
model_checkpoint = ModelCheckpoint('bestfrac_model.keras', save_best_only=True, monitor='val_loss')

In [18]:
# Train the model with callbacks
history = model.fit(combined_data, labels_one_hot, validation_split=0.2, epochs=25, batch_size=32,
                    callbacks=[early_stopping, model_checkpoint])

Epoch 1/25
675/675 ━━━━━━━━━━━━━━━━━━━━ 490s 670ms/step - accuracy: 0.0999 - loss: 2.7300 - val_accuracy: 0.0972 - val_loss: 2.9586
Epoch 2/25
675/675 ━━━━━━━━━━━━━━━━━━━━ 443s 656ms/step - accuracy: 0.1056 - loss: 2.5161 - val_accuracy: 0.1026 - val_loss: 2.6222
Epoch 3/25
675/675 ━━━━━━━━━━━━━━━━━━━━ 442s 655ms/step - accuracy: 0.1110 - loss: 2.4868 - val_accuracy: 0.0996 - val_loss: 2.7664
Epoch 4/25
675/675 ━━━━━━━━━━━━━━━━━━━━ 440s 651ms/step - accuracy: 0.1175 - loss: 2.4818 - val_accuracy: 0.0994 - val_loss: 2.4786
Epoch 5/25
675/675 ━━━━━━━━━━━━━━━━━━━━ 436s 645ms/step - accuracy: 0.1241 - loss: 2.4094 - val_accuracy: 0.1015 - val_loss: 2.4129
Epoch 6/25
675/675 ━━━━━━━━━━━━━━━━━━━━ 436s 646ms/step - accuracy: 0.1282 - loss: 2.3773 - val_accuracy: 0.1087 - val_loss: 2.4208
Epoch 7/25
675/675 ━━━━━━━━━━━━━━━━━━━━ 435s 645ms/step - accuracy: 0.1484 - loss: 2.3360 - val_accuracy: 0.0959 - val_loss: 2.6372
Epoch 8/25
675/675 ━━━━━━━━━━━━━━━━━━━━ 433s 641ms/step - accuracy: 0.1699 -

In [ ]:
from tensorflow.keras.models import load_model

# Load the last best model checkpoint saved
model = load_model('bestfrac_model.keras')

# Re-compile the model if necessary
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Callbacks for resuming training
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
model_checkpoint = ModelCheckpoint('bestfrac_model.keras', save_best_only=True, monitor='val_loss')

# Resume training
history = model.fit(
    combined_data, labels_one_hot, 
    validation_split=0.2, 
    initial_epoch=15,  # Start from epoch 15
    epochs=25,         # Continue up to the 25th epoch
    batch_size=32, 
    callbacks=[early_stopping, model_checkpoint]
)


Epoch 16/25
675/675 ━━━━━━━━━━━━━━━━━━━━ 770s 1s/step - accuracy: 0.1287 - loss: 2.4493 - val_accuracy: 0.1022 - val_loss: 2.5540
Epoch 17/25
675/675 ━━━━━━━━━━━━━━━━━━━━ 680s 1s/step - accuracy: 0.1220 - loss: 2.4118 - val_accuracy: 0.0967 - val_loss: 2.4242
Epoch 18/25
328/675 ━━━━━━━━━━━━━━━━━━━━ 2:08:30 22s/step - accuracy: 0.1521 - loss: 2.3389

In [ ]:
import matplotlib.pyplot as plt

plt.plot(history.history['accuracy'], label='accuracy')
plt.plot(history.history['val_accuracy'], label='val_accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()
plt.show()

In [2]:
from tensorflow.keras.models import load_model

# Load the last best model checkpoint saved
model = load_model('bestfrac_model.keras')

In [3]:
# Evaluate the model
test_loss, test_accuracy = model.evaluate(combined_data, labels_one_hot, verbose=1)
print(f"Test Accuracy: {test_accuracy}")
print(f"Test Loss: {test_loss}")


NameError: name 'combined_data' is not defined